In [1]:
import os, torchaudio                   # type: ignore

root = os.path.expanduser("~/.cache")

ds = torchaudio.datasets.LIBRISPEECH(root=root, url="test-clean", download=True)

audio, sr, text, *_ = ds[0]

In [2]:
import whisper

model = whisper.load_model("base")

waveform = audio.squeeze().numpy()

result = model.transcribe(waveform, fp16=False)

result["text"]

' He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered flour-fatten sauce.'

In [6]:
from ollama import chat                 # type: ignore
from ollama import ChatResponse         # type: ignore

system_prompt = "You are a helpful, polite, and clear assistant in all your responses."

response: ChatResponse = chat(model='phi3:instruct', messages=[
    { "role": "system", "content": system_prompt },
    { 'role': 'user', 'content': result["text"] },
])

print(response['message']['content'])

He eagerly anticipated a hearty homemade stew as the main course of his meal. He envisioned chunks of tender turnips simmering alongside earthy carrots and potatoes that had seen better days but still possessed flavorful depth due to their slight bruising, all complemented by rich mutton pieces generously cooked in a velvety sauce thickened with pepper-infused flour. The dish promised warmth not just for his belly but also an invigorating experience that connected him back to traditional comfort foods of the past.


In [7]:
text_prompt = response['message']['content']

In [10]:
from transformers import BarkModel, AutoProcessor
import torch

device = "cuda"

model = BarkModel.from_pretrained("suno/bark-small").to(device)
processor = AutoProcessor.from_pretrained("suno/bark")

inputs = processor(text_prompt, return_tensors="pt").to(device)

with torch.no_grad():
    speech_output = model.generate(**inputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [12]:
import scipy.io.wavfile as wav

audio = speech_output.cpu().numpy().squeeze()
wav.write("output.wav", model.generation_config.sample_rate, audio)

In [15]:
from pydub import AudioSegment

AudioSegment.from_wav("output.wav").export("output.mp3", format="mp3")

<_io.BufferedRandom name='output.mp3'>